# 파이프라인을 활용한 Iris 분류 노트북

이 Jupyter Notebook은 `scikit-learn`의 `load_iris` 데이터셋을 사용하여 **파이프라인(`Pipeline`)**을 구성하고, 데이터 스케일링(`StandardScaler`)과 `SVC` (Support Vector Classifier) 모델 학습을 통합하는 과정을 보여줍니다. 마지막으로, 모델의 성능을 `classification_report`를 통해 평가합니다.

**파이프라인(`Pipeline`) 개요:**
* **전처리 및 모델 통합**: 여러 전처리 단계와 최종 모델을 하나의 객체로 묶어줍니다.
* **코드 간소화**: 반복적인 전처리 코드를 줄이고 워크플로우를 명확하게 만듭니다.
* **데이터 누수 방지**: 훈련 데이터에만 전처리를 적용하고 테스트 데이터에는 훈련 데이터의 통계량으로 변환하는 등 데이터 누수를 효과적으로 방지합니다.

---
## 1. 라이브러리 임포트

필요한 라이브러리들을 임포트합니다.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline # 파이프라인
from sklearn.svm import SVC
from sklearn.metrics import classification_report # 분류 레포트

---
## 2. 데이터 로드 및 확인

Iris 데이터셋을 로드하고 특성 데이터(`X`)와 타겟 데이터(`y`)의 형태를 확인합니다. Iris 데이터셋은 붓꽃의 세 가지 종을 분류하는 데 사용되는 대표적인 분류 데이터셋입니다.

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

print("Iris 데이터셋 정보:")
print(f"특성 개수: {X.shape[1]}")
print(f"샘플 개수: {X.shape[0]}")
print(f"클래스: {iris.target_names}")

---
## 3. 데이터 분할

데이터를 훈련 세트와 테스트 세트로 분할합니다. 테스트 세트의 크기는 전체 데이터의 20%로 설정합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=1234)

---
## 4. 파이프라인 정의 및 학습

데이터 스케일링(`StandardScaler`)과 `SVC` 모델을 포함하는 파이프라인을 정의하고 훈련 데이터에 학습시킵니다. 파이프라인을 사용하면 전처리 단계와 모델 학습 단계를 한 번의 `fit` 호출로 처리할 수 있습니다.

In [ ]:
# 파이프라인 정의하기
pipeline = Pipeline(
    [
        ('scaler', StandardScaler()), # 데이터 스케일링
        ('svc', SVC(kernel='rbf', C=1.0, gamma='scale')) # SVC 모델
    ]
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

---
## 5. 모델 성능 평가

학습된 파이프라인 모델의 성능을 `classification_report`를 사용하여 평가합니다. 이 보고서는 정밀도(precision), 재현율(recall), F1-점수(F1-score) 및 각 클래스별 지원(support)을 포함합니다.

In [ ]:
# 성능 평가
print("분류 보고서:")
print(classification_report(y_test, y_pred))

---
## 6. GridSearchCV와 파이프라인 활용 (추가 학습)

파이프라인은 `GridSearchCV`와 같은 하이퍼파라미터 튜닝 도구와 함께 사용될 때 매우 강력합니다. `GridSearchCV`의 `estimator`에 파이프라인 객체를 전달하여 전처리 단계와 모델의 하이퍼파라미터를 동시에 최적화할 수 있습니다.

```python
from sklearn.model_selection import GridSearchCV

# 파이프라인 정의
pipeline_grid = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(random_state=1234))
])

# GridSearchCV를 위한 파라미터 그리드
# 파이프라인 단계의 하이퍼파라미터는 '단계이름__파라미터이름' 형식으로 지정합니다.
param_grid = {
    'scaler': [StandardScaler(), MinMaxScaler()], # 다른 스케일러 시도
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 'scale', 'auto'],
    'svc__kernel': ['rbf', 'linear']
}

# GridSearchCV 실행
grid_search = GridSearchCV(pipeline_grid, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print("\n최적 파라미터:", grid_search.best_params_)
print("최고 교차 검증 점수:", grid_search.best_score_)

# 최적 모델로 예측 및 평가
best_pipeline = grid_search.best_estimator_
y_pred_best = best_pipeline.predict(X_test)
print("\n최적 모델 분류 보고서:")
print(classification_report(y_test, y_pred_best))
```